## Homework: Vector Search (Version 2)

In this homework, we'll experiment with vector search with and without Elasticsearch.

### Q1. Getting the embeddings model

First we will get the embeddings model multi-qa-distilbert-cos-v1 from the [Sentence Transformer library](https://www.sbert.net/docs/sentence_transformer/pretrained_models.html#model-overview)

```
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer(model_name)
```

Create the embedding for this user question:

```
user_question = "I just discovered the course. Can I still join it?"
```

What is the first value of the resulting vector?

* -0.24
* -0.04
* 0.07
* 0.27

In [1]:
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer("multi-qa-distilbert-cos-v1")

In [2]:
user_question = "I just discovered the course. Can I still join it?"

embeddings = embedding_model.encode(user_question)
embeddings[0]

0.078222655

### A1. 0.07 is closest

For question 3

In [3]:
v = embeddings

### Prepare the documents

Now we will create the embeddings for the documents.

Load the documents with ids that we prepared in the module:



In [4]:
import requests

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

In [5]:
len(documents)

948

We will use only a subset of the questions - the questions for "machine-learning-zoomcamp".  After filtering, you should have only 375 documents.

In [6]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp',
 'id': 'c02e79ef'}

In [7]:
filtered_docs = [d for d in documents if d['course'] == 'machine-learning-zoomcamp']
len(filtered_docs)

375

### Q2. Creating the embeddings

Now for each document, we will create an embedding for both the question and answer fields.

We want to put all of them into a single matrix X:

* Create a list of embeddings
* Iterate over each document
* qa_text = f'{question} {text}'
* compute the embedding for qa_text, append it to embeddings
* at the end, let X = np.array(embeddings) (import numpy as np)

What is the shape of X? ( X.shape ). Include the parentheses.

In [8]:
import numpy as np

from sentence_transformers import SentenceTransformer
model = SentenceTransformer("multi-qa-distilbert-cos-v1")

# creating a dense vector using the pre-trained model
embeddings = []
for doc in filtered_docs:
    question = doc['question']
    text = doc['text']
    qa_text = f'{question} {text}'
    embeddings.append(model.encode(qa_text).tolist())

In [9]:
X = np.array(embeddings)
X.shape

(375, 768)

### A2. The shape of X is (375,768)

### Q3. Search

We have the embeddings and the query vector.  Now let's compute the cosine similarity between the vector from Q1 (let's call it v) and the matrix from Q2.


The vectors returned from the embedding model are already normalized (you can check it by computing a dot product of a vector with itself - it should return 1.0).  This means that in order to compute the cosine similarity, it's sufficient to multiply the matrix X by the vectort v:

```
scores = X.dot(v)
```

What's the highest score in the results?

* 65.0
* 6.5
* 0.65
* 0.065

In [10]:
scores = X.dot(v)

In [11]:
scores.max()

0.6506573240979582

### A3. The maximum score is 0.65

### Vector search

We can now compute the similarity between a query vector and all the embeddings.

Let's use this to implement our own vector search

In [12]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

search_engine = VectorSearchEngine(documents=documents, embeddings=X)
search_engine.search(v, num_results=5)

[{'text': 'You can find the latest and up-to-date deadlines here: https://docs.google.com/spreadsheets/d/e/2PACX-1vQACMLuutV5rvXg5qICuJGL-yZqIV0FBD84CxPdC5eZHf8TfzB-CJT_3Mo7U7oGVTXmSihPgQxuuoku/pubhtml\nAlso, take note of Announcements from @Au-Tomator for any extensions or other news. Or, the form may also show the updated deadline, if Instructor(s) has updated it.',
  'section': 'General course-related questions',
  'question': 'Homework - What are homework and project deadlines?',
  'course': 'data-engineering-zoomcamp',
  'id': 'a1daf537'},
 {'text': 'After you submit your homework it will be graded based on the amount of questions in a particular homework. You can see how many points you have right on the page of the homework up top. Additionally in the leaderboard you will find the sum of all points you’ve earned - points for Homeworks, FAQs and Learning in Public. If homework is clear, others work as follows: if you submit something to FAQ, you get one point, for each learning i

If you don't understand how the search function works:

* Ask ChatGTP or any other LLM of your choice to explain the code
* Check our pre-course workshop about implementing a search engine here

(Note: you can replace argsort with argpartition to make it a lot faster)

### ___Will need to complete later___

### Q4. Hit-rate for our search engine

Let's evaluate the performance of our own search engine. We will use the hit rate metric for evaluation.

Hit Rate (HR) or Recall at k:
```
    Measures the proportion of queries for which at least one relevant document is retrieved in the top k results.
    Formula: HR@k = (Number of queries with at least one relevant document in top k) / |Q|
```

First, load the ground truth dataset:

In [13]:
import pandas as pd

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

Now use the code from the module to calculate the hit rate of VectorSearchEngine with num_results=5.

In [15]:
next(iter(ground_truth))

{'question': 'Where can I sign up for the course?',
 'course': 'machine-learning-zoomcamp',
 'document': '0227b872'}

In [16]:
len(ground_truth)

1830

In [22]:
from tqdm.auto import tqdm
relevance_total = []

for q in tqdm(ground_truth):
    doc_id = q['document']
    results = search_engine.search(v, num_results=5)
    relevance = [d['id'] == doc_id for d in results]
    relevance_total.append(relevance)

  0%|          | 0/1830 [00:00<?, ?it/s]

In [23]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

In [24]:
hit_rate(relevance_total)

0.0

What did you get?

* 0.93
* 0.73
* 0.53
* 0.33

### A4.

### Q5. Indexing with Elasticsearch

Now let's index these documents with elasticsearch

```
    Create the index with the same settings as in the module (but change the dimensions)
    Index the embeddings (note: you've already computed them)
```
After indexing, let's perform the search of the same query from Q1.

What's the ID of the document with the highest score?

### Q6. Hit-rate for Elasticsearch

The search engine we used in Q4 computed the similarity between the query and ALL the vectors in our database. Usually this is not practical, as we may have a lot of data.

Elasticsearch uses approximate techniques to make it faster.

Let's evaluate how worse the results are when we switch from exact search (as in Q4) to approximate search with Elastic.

What's hit rate for our dataset for Elastic?

* 0.93
* 0.73
* 0.53
* 0.33